In [1]:
from fake_useragent import UserAgent
import time
import random
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import csv
import json

### Get total number of pages

In [2]:
def get_total_pages(url):
    try:
        html_text = requests.get(url,headers={'User-Agent': ua.random}).text
        bs1 = BeautifulSoup(html_text)
        temp = bs1.find("div",{'class':"lemon--div__373c0__1mboc padding-b2__373c0__34gV1 border-color--default__373c0__3-ifU text-align--center__373c0__2n2yQ"})
        temp_text = temp.find('span',{'class':"lemon--span__373c0__3997G text__373c0__2Kxyz text-color--black-extra-light__373c0__2OyzO text-align--left__373c0__2XGa-"}).text
        total_pages = int(temp_text.split(" ")[-1])
        return total_pages
    except Exception:
        return -1

In [3]:
def get_shop_list(url):
    url = 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Atlanta%2C+GA%2C+United+States&ns=1'
    total_pages = get_total_pages(url)
    df = pd.DataFrame(columns=["biz_name","biz_url"]) 
    for i in range(0,total_pages):
        page_url = url + "&start={}".format(i * 30)
        page_text = requests.get(page_url,headers={'User-Agent': ua.random}).text
        page_bs = BeautifulSoup(page_text)
        count = 0
        for j in page_bs.find_all('a',{'class':"lemon--a__373c0__IEZFH link__373c0__1G70M link-color--inherit__373c0__3dzpk link-size--inherit__373c0__1VFlE"},href=re.compile('/biz/')):
            count = count + 1
            link = "https://www.yelp.com" + j.get("href")
            bizname = j.get("name")
            biz_index = i * 30 + count
            df.loc[biz_index] = [bizname,link]
    return df
        


In [4]:
df = get_shop_list('https://www.yelp.com/search?find_desc=Restaurants&find_loc=Atlanta%2C+GA%2C+United+States&ns=1')
df.to_csv("shoplink2.csv",header=False,index=True)


### Total Page of reviews

In [6]:
def get_total_page_reviews(url):
    try:
        ua = UserAgent()
        html_text = requests.get(url,headers={'User-Agent': ua.random}).text
        bs = BeautifulSoup(html_text)
        temp = bs.find("div",{"class":"lemon--div__373c0__1mboc pagination__373c0__1NjN5 u-padding-t2 border--bottom__373c0__uPbXS border-color--default__373c0__2oFDT"})
        total_page_reviews = temp.find("span",{"class":"lemon--span__373c0__3997G text__373c0__2pB8f text-color--inherit__373c0__w_15m text-align--left__373c0__2pnx_"})
        total_page_reviews = str(total_page_reviews.text).split(" ")[-1]
        return int(total_page_reviews)
    except Exception:
        return -1

In [7]:
def get_reviews(re_url,count_i):
    ua = UserAgent()
    total_page_reviews = get_total_page_reviews(re_url)
    review_info = pd.DataFrame(columns=["biz_id","biz_url","user_id","user_name","friends","number_reviews","photos","area","date","elite","rating"])
    html_text = requests.get(re_url,headers={'User-Agent': ua.random}).text
    bs = BeautifulSoup(html_text)
    temp = bs.find("head")
    z = temp.find("meta", attrs={'name':'yelp-biz-id'})
    biz_id = z['content']
    count = count_i
    for ii in range(0,total_page_reviews):
        page_url = re_url + "&start={}".format(ii * 20)
        page_text = requests.get(page_url,headers={'User-Agent': ua.random}).text
        page_bs = BeautifulSoup(page_text)
        container = page_bs.find_all("li",{"class":"lemon--li__373c0__1r9wz u-space-b3 u-padding-b3 border--bottom__373c0__uPbXS border-color--default__373c0__2oFDT"})
        for i in container:
            user_info =  i.find("a",{"class":"lemon--a__373c0__IEZFH link__373c0__29943 link-color--inherit__373c0__15ymx link-size--inherit__373c0__2JXk5"},href=re.compile("^/user_details"))
            user_name = user_info.text
            user_id = str(user_info.get("href")).split("=")[-1]
            # get tthe attributes of user
            friends = photos = number_reviews = None
            for index,j in enumerate(i.find_all("span",{"class":"lemon--span__373c0__3997G text__373c0__2pB8f text-color--normal__373c0__K_MKN text-align--left__373c0__2pnx_ text-size--small__373c0__3SGMi"})):
                if index == 0:
                    friends = j.text
                elif index == 1:
                    number_reviews = j.text
                elif index == 2:
                    photos = j.text

            # get the area
            area = None
            for index,j in enumerate(i.find_all("span",{"class":"lemon--span__373c0__3997G text__373c0__2pB8f text-color--normal__373c0__K_MKN text-align--left__373c0__2pnx_ text-weight--bold__373c0__3HYJa text-size--small__373c0__3SGMi"})):
                area = j.text

            # get the date
            date_posted = None
            date_posted = i.find("span",{"class":"lemon--span__373c0__3997G text__373c0__2pB8f text-color--mid__373c0__3G312 text-align--left__373c0__2pnx_"})
            if date_posted != None:
                date = date_posted.text

            # get rating
            rating = None
            temp_rating = i.find("span",{"class":"lemon--span__373c0__3997G display--inline__373c0__3JgLR border-color--default__373c0__MD4Lj"})
            if temp_rating != None:
                rating = temp_rating.find(name="div",attrs={"aria-label":True}).get("aria-label")[0]

            # elite or not
            elite = None
            temp_elite = i.find("p",{"class":"lemon--p__373c0__3Qnnj text__373c0__2pB8f text-color--inherit__373c0__w_15m text-align--left__373c0__2pnx_ text-size--inherit__373c0__2gFQ3"})
            if temp_elite != None:
                elite = temp_elite.text
            
            review_info.loc[count] = [biz_id,re_url,user_id,user_name,friends,number_reviews,photos,area,date,elite,rating]
            count = count + 1
            time.sleep(random.random()+1)
    return review_info,count

In [8]:
data = pd.read_csv("shoplink.csv",header=None)
data.columns = ["index","biz-name","biz-link"]
data = data.drop(["index"],axis=1)
count_i = 0
for i in range(6,10):
    link = data.iloc[i][1]
    count = count_i
    print(link)
    reviews,count_i = get_reviews(link,count)
    reviews.to_csv('reviews3.csv', mode='a+', header=False)

https://www.yelp.com/biz/two-urban-licks-atlanta?osq=Restaurants


KeyboardInterrupt: 